In [136]:
import re
###REWORKER FOR INITIAL FILES
def fix_csv(csv_name):
    csv = open(csv_name, 'r', encoding="UTF-8")
    newcsv = open(csv_name[:-4] + "_R.csv", 'w', encoding="UTF-8")
    names = csv.readline().strip()
    newcsv.write(names.replace(' ', ',') + "\n")
    for line in csv.readlines():
        check = re.search(r'"(.*)"', line)
        if check:
            for match in check.groups():
                line = re.sub(match, match.replace(' ', '_'), line)
        splited_list = line.split(' ')[1:]
        if '' in splited_list:
           splited_list[splited_list.index('')] = 'NaN'
        newcsv.write(','.join(splited_list))
    csv.close()
    newcsv.close()

In [140]:
fix_csv('cinema_sessions.csv')
fix_csv('titanic_with_labels.csv')  

In [166]:
from os.path import exists
import numpy as np
import pandas as pd

SEX_MALE_DEFF = ["M", "m", "М", "м"]
SEX_FEM_DEFF = ["F", "ж", "Ж", "f"]
DRINK_NORMALIZED_VALUE = 20

if not exists('cinema_sessions_R.csv'):
    fix_csv('cinema_sessions.csv')
    fix_csv('titanic_with_labels.csv')    
sess = pd.read_csv('cinema_sessions_R.csv', encoding="UTF-8")
tit = pd.read_csv('titanic_with_labels_R.csv', encoding="UTF-8")

#Fix genders
tit.sex = tit.sex.replace(SEX_FEM_DEFF, 1).replace(SEX_MALE_DEFF, 0)
tit = tit[tit.sex.apply(str).str.isdigit()]

#Fix row_numbers
tit.row_number = tit.row_number.replace(regex=['NaN'],value=np.nan).fillna(tit.row_number.max())

#Fix liters
litersDrunkNormalized = tit.liters_drunk[(tit.liters_drunk >= 0) & (tit.liters_drunk <= DRINK_NORMALIZED_VALUE)]
tit.loc[(tit.liters_drunk <= 0) | (tit.liters_drunk >= DRINK_NORMALIZED_VALUE), 'liters_drunk'] = int(litersDrunkNormalized.mean())



,sex,row_number,liters_drunk,age,drink,check_number,label
0,1,79.0,2,66.0,Cola,3116.0,0.0
1,0,23.0,2,10.0,Fanta,8118.0,0.0
2,0,92.0,2,18.0,Fanta,2264.0,1.0
3,1,52.0,5,67.0,Beerbeer,7640.0,1.0
4,1,57.0,2,27.0,Beerbeer,8764.0,0.0
...,...,...,...,...,...,...,...
9990,1,100.0,3,15.0,Water,1022.0,1.0
9992,0,15.0,3,62.0,Strong_beer,6510.0,0.0
9994,1,11.0,3,52.0,Strong_beer,9510.0,1.0
9995,0,96.0,5,72.0,Beerbeer,9554.0,0.0
